## Time Series Forecasting .


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.api import VAR


In [2]:
df=pd.read_csv("appliance_energy_usage.csv",index_col=0,parse_dates=True)

## T_out= temperature outside, press_mm_hg =pressure , RH_out= Humidity 

In [3]:
df

,Appliances,lights,T_out,Press_mm_hg,RH_out,Windspeed,Tdewpoint,Visibility
date,,,,,,,,
2016-01-11 17:00:00,60,30,6.600000,733.5,92.000000,7.000000,5.300000,63.000000
2016-01-11 17:10:00,60,30,6.483333,733.6,92.000000,6.666667,5.200000,59.166667
2016-01-11 17:20:00,50,30,6.366667,733.7,92.000000,6.333333,5.100000,55.333333
2016-01-11 17:30:00,50,40,6.250000,733.8,92.000000,6.000000,5.000000,51.500000
2016-01-11 17:40:00,60,40,6.133333,733.9,92.000000,5.666667,4.900000,47.666667
...,...,...,...,...,...,...,...,...
2016-05-27 17:20:00,100,0,22.733333,755.2,55.666667,3.333333,13.333333,23.666667
2016-05-27 17:30:00,90,0,22.600000,755.2,56.000000,3.500000,13.300000,24.500000
2016-05-27 17:40:00,270,10,22.466667,755.2,56.333333,3.666667,13.266667,25.333333


In [4]:
df=df.resample("1H").mean()

In [5]:
df

,Appliances,lights,T_out,Press_mm_hg,RH_out,Windspeed,Tdewpoint,Visibility
date,,,,,,,,
2016-01-11 17:00:00,55.000000,35.000000,6.308333,733.750000,92.000000,6.166667,5.050000,53.416667
2016-01-11 18:00:00,176.666667,51.666667,5.941667,734.266667,91.583333,5.416667,4.658333,40.000000
2016-01-11 19:00:00,173.333333,25.000000,6.000000,734.791667,89.750000,6.000000,4.391667,40.000000
2016-01-11 20:00:00,125.000000,35.000000,6.000000,735.283333,87.583333,6.000000,4.016667,40.000000
2016-01-11 21:00:00,103.333333,23.333333,5.833333,735.566667,87.416667,6.000000,3.816667,40.000000
...,...,...,...,...,...,...,...,...
2016-05-27 14:00:00,101.666667,1.666667,21.916667,755.800000,59.000000,2.000000,13.475000,21.583333
2016-05-27 15:00:00,76.666667,0.000000,22.216667,755.675000,57.333333,2.000000,13.258333,21.833333
2016-05-27 16:00:00,135.000000,0.000000,22.883333,755.375000,55.000000,2.416667,13.283333,22.583333


In [6]:
df.corr()

,Appliances,lights,T_out,Press_mm_hg,RH_out,Windspeed,Tdewpoint,Visibility
Appliances,1.000000,0.261794,0.127453,-0.044034,-0.194515,0.112477,0.021073,-0.003380
lights,0.261794,1.000000,-0.085036,-0.012278,0.078925,0.069831,-0.041437,0.023114
T_out,0.127453,-0.085036,1.000000,-0.143233,-0.574353,0.193064,0.792621,-0.081857
Press_mm_hg,-0.044034,-0.012278,-0.143233,1.000000,-0.092555,-0.236864,-0.244271,0.042413
RH_out,-0.194515,0.078925,-0.574353,-0.092555,1.000000,-0.175334,0.033267,0.087726
Windspeed,0.112477,0.069831,0.193064,-0.236864,-0.175334,1.000000,0.127627,-0.007935
Tdewpoint,0.021073,-0.041437,0.792621,-0.244271,0.033267,0.127627,1.000000,-0.045001
Visibility,-0.003380,0.023114,-0.081857,0.042413,0.087726,-0.007935,-0.045001,1.000000


## checking stationarity

**h0: series is not stationary**\
**h1: series is stationary**\
**ADF test**

In [7]:
for i in range(len(df.columns)):
    result=adfuller(df[df.columns[i]])
    
    if result[1]>0.05:
        print("series is not stationary")
    else:
        print("series is stationary")

series is stationary
series is stationary
series is stationary
series is stationary
series is stationary
series is stationary
series is stationary
series is stationary


 ## grangercasualitytest

In [11]:
max_lags=8
y="Appliances"

In [12]:
for i in range(len(df.columns)-1):
    results=grangercausalitytests(df[[y,df.columns[i+1]]],max_lags,verbose=False)
    p_values=[round(results[i+1][0]['ssr_ftest'][1],4) for i in range(max_lags)]
    print('column - {}:P_values - {}'.format(df.columns[i+1],p_values))
    

C:\Users\VAIBHAV\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


column - lights:P_values - [0.1451, 0.0005, 0.0001, 0.0002, 0.0001, 0.0001, 0.0006, 0.0012]
column - T_out:P_values - [0.002, 0.0001, 0.0001, 0.0, 0.0, 0.0, 0.0, 0.0]


C:\Users\VAIBHAV\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
C:\Users\VAIBHAV\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


column - Press_mm_hg:P_values - [0.0985, 0.1956, 0.0351, 0.0016, 0.0, 0.0001, 0.0, 0.0]
column - RH_out:P_values - [0.0, 0.0004, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0]


C:\Users\VAIBHAV\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
C:\Users\VAIBHAV\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


column - Windspeed:P_values - [0.0004, 0.0041, 0.0094, 0.0082, 0.0163, 0.0119, 0.0009, 0.0002]
column - Tdewpoint:P_values - [0.4348, 0.4494, 0.5005, 0.3887, 0.05, 0.0694, 0.0992, 0.1152]


C:\Users\VAIBHAV\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
C:\Users\VAIBHAV\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


column - Visibility:P_values - [0.7071, 0.5372, 0.7338, 0.8265, 0.544, 0.3324, 0.2521, 0.3465]


In [13]:
df_input=df[['Appliances','T_out','Windspeed']]

In [14]:
df_input

,Appliances,T_out,Windspeed
date,,,
2016-01-11 17:00:00,55.000000,6.308333,6.166667
2016-01-11 18:00:00,176.666667,5.941667,5.416667
2016-01-11 19:00:00,173.333333,6.000000,6.000000
2016-01-11 20:00:00,125.000000,6.000000,6.000000
2016-01-11 21:00:00,103.333333,5.833333,6.000000
...,...,...,...
2016-05-27 14:00:00,101.666667,21.916667,2.000000
2016-05-27 15:00:00,76.666667,22.216667,2.000000
2016-05-27 16:00:00,135.000000,22.883333,2.416667


In [16]:
df_train=df_input[:int(0.9*(len(df_input)))]
df_test=df_input[int(0.9*(len(df_input))):]

In [17]:
df_train.shape

(2961, 3)

In [18]:
df_test.shape

(329, 3)

In [20]:
model=VAR(df_train,freq='1H')
model.select_order(40).summary()

,AIC,BIC,FPE,HQIC
0,13.81,13.82,9.993e+05,13.82
1,7.135,7.159,1255.,7.144
2,6.147,6.190,467.2,6.162
3,6.028,6.090,415.1,6.051
4,5.968,6.048,390.7,5.997
5,5.914,6.013,370.3,5.950
6,5.899,6.016,364.7,5.941
7,5.860,5.995*,350.8,5.909
8,5.849,6.003,347.0,5.905
9,5.834,6.006,341.9,5.896


In [22]:
model=VAR(df_train,freq='1H')
results=model.fit(7)
print(results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 11, Sep, 2023
Time:                     07:30:56
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    5.98610
Nobs:                     2954.00    HQIC:                   5.90043
Log likelihood:          -21152.4    FPE:                    348.017
AIC:                      5.85225    Det(Omega_mle):         340.356
--------------------------------------------------------------------
Results for equation Appliances
                   coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------
const                37.865935         3.448219           10.981           0.000
L1.Appliances         0.451436         0.018470           24.442           0.000
L1.T_out              2.328238         2.534185        

In [23]:
lag=results.k_ar

In [24]:
results.forecast(df_train.values[-lag:],steps=5)

array([[44.34598018,  8.92651619,  3.99333186],
       [33.70923187,  8.81719814,  4.13871016],
       [38.32344216,  9.01953267,  4.188856  ],
       [53.39756915,  9.3356185 ,  4.31429082],
       [60.93798924,  9.56974211,  4.40093836]])

In [25]:
df_test[0:5]

,Appliances,T_out,Windspeed
date,,,
2016-05-14 02:00:00,60.000000,8.891667,3.166667
2016-05-14 03:00:00,60.000000,8.725000,2.833333
2016-05-14 04:00:00,60.000000,8.775000,4.416667
2016-05-14 05:00:00,56.666667,8.683333,5.000000
2016-05-14 06:00:00,63.333333,8.716667,5.000000


In [26]:
df_coeff=pd.DataFrame([results.params['Appliances'],results.pvalues['Appliances']]).T

In [27]:
df_coeff

,Appliances,Appliances
const,37.865935,4.701032e-28
L1.Appliances,0.451436,6.138160e-132
L1.T_out,2.328238,3.582355e-01
L1.Windspeed,2.268735,2.564126e-01
L2.Appliances,0.222279,4.255512e-28
L2.T_out,-4.372626,4.302427e-01
L2.Windspeed,-1.481847,6.603327e-01
L3.Appliances,-0.019489,3.454670e-01
L3.T_out,5.606923,3.952787e-01
L3.Windspeed,0.184615,9.598961e-01


In [28]:
df_coeff.columns=['coeff','pval']

In [29]:
df_coeff.query('pval<0.05')
np.where(df_coeff['pval']<0.05,1,0)

array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [31]:
df_coeff['valid']=np.where(df_coeff['pval']<0.05,1,0)

In [32]:
df_coeff

,coeff,pval,valid
const,37.865935,4.701032e-28,1
L1.Appliances,0.451436,6.138160e-132,1
L1.T_out,2.328238,3.582355e-01,0
L1.Windspeed,2.268735,2.564126e-01,0
L2.Appliances,0.222279,4.255512e-28,1
L2.T_out,-4.372626,4.302427e-01,0
L2.Windspeed,-1.481847,6.603327e-01,0
L3.Appliances,-0.019489,3.454670e-01,0
L3.T_out,5.606923,3.952787e-01,0
L3.Windspeed,0.184615,9.598961e-01,0


In [36]:
coeff_arr=df_coeff['coeff'][1:].values

In [37]:
coeff_arr.shape

(21,)

In [38]:
coeff_arr

array([ 0.45143647,  2.32823846,  2.26873466,  0.2222793 , -4.37262576,
       -1.48184722, -0.01948858,  5.60692324,  0.18461514, -0.04369054,
       -5.43530357, -0.08685622,  0.00738616,  5.17191268, -1.99030973,
       -0.01683822,  3.22703425,  4.97164661, -0.01091878, -6.77733848,
       -2.97259279])

In [39]:
df_train[-lag:]

,Appliances,T_out,Windspeed
date,,,
2016-05-13 19:00:00,208.333333,19.333333,9.000000
2016-05-13 20:00:00,120.000000,17.066667,8.166667
2016-05-13 21:00:00,63.333333,14.116667,6.583333
2016-05-13 22:00:00,55.000000,11.850000,6.833333
2016-05-13 23:00:00,61.666667,10.466667,7.583333
2016-05-14 00:00:00,58.333333,9.791667,5.750000
2016-05-14 01:00:00,63.333333,9.333333,4.000000


In [40]:
in_arr=df_train[-lag:][::-1].stack().to_frame().T.values

In [41]:
in_arr.shape

(1, 21)

In [42]:
in_arr

array([[ 63.33333333,   9.33333333,   4.        ,  58.33333333,
          9.79166667,   5.75      ,  61.66666667,  10.46666667,
          7.58333333,  55.        ,  11.85      ,   6.83333333,
         63.33333333,  14.11666667,   6.58333333, 120.        ,
         17.06666667,   8.16666667, 208.33333333,  19.33333333,
          9.        ]])

In [44]:
np.dot(in_arr,coeff_arr)+df_coeff['coeff'][:1].values

array([44.34598018])